In [ ]:
#Need to create list of synonyms and antonyms from words found on Thesaurus.com and Word.net
#First thought is to create a list of words of interest that can be queried directly by our script 
#Need to also create a list of list of the words that we used to create the syn-ant query list.

In [ ]:
#For Thesaurus.com (T.com)
import numpy as np
import pandas as pd
import selenium
from selenium import webdriver
import bs4
from bs4 import BeautifulSoup as bs
import time
import re
import nltk
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from collections import defaultdict
import requests
import time


In [ ]:
useful_words = ['corrosion', 'flammable', 'resistant', 'retardate', 'malleable']
syn_list = []
ant_list = []

start = time.time() #To determine scalability. Times are inconsistent.

for word in useful_words: #pull words from a premade list to query T.com
    driver = webdriver.Chrome()
    driver.get('https://www.thesaurus.com/browse/' + word)
    time.sleep(2)
    
    html_doc = driver.page_source # stores the source HTML code in the driver's page_source attribute
   
    #Issue with next two lines is that we are also picking up "Related Terms" synonyms and antonyms.
    #Dirty fix is to keep only the 5 ant and syn results from a query after it has been cleaned.
    syn = re.findall(r'"similarity":"100","isInformal":"0","isVulgar":null,"term":"(\S*?)","targetTerm"', html_doc) 
    ant = re.findall(r'"similarity":"-100","isInformal":"0","isVulgar":null,"term":"(\S*?)","targetTerm"', html_doc)
    
    syn_list.append(syn)
    ant_list.append(ant)
    
    driver.quit()
    
end = time.time()
print(end - start)
    #To retrieve "Related Terms" from a query to add to list
    #"entry":"corrosion","targetTerm":"canker","targetSlug" 



In [ ]:
#DataFrame creation based off amount of 'useful_words'
test_array = np.empty((len(useful_words),3))
test_array_df = pd.DataFrame(data=test_array[0:,:], columns = {'Term', 'Synonyms', 'Antonyms'})

#Creation of raw list of lists of syn/ant for each 'useful_word'
raw_syn_list = [[] for i in range(len(useful_words))]
raw_ant_list = [[] for i in range(len(useful_words))]


In [ ]:
#Clean up scraped data (remove repetitions, need to find out how many syn/ant we want to keep)
#Populate raw list of lists with cleaned syn/ant for each'useful_word'
for i in range(len(useful_words)):
    for word in syn_list[i]:
        if word in raw_syn_list[i]:
            continue
        else:
            raw_syn_list[i].append(word)
    
for i in range(len(useful_words)):
    for word in ant_list[i]:
        if word in raw_ant_list[i]:
            continue
        else:
            raw_ant_list[i].append(word)

In [ ]:
#Populate DataFrame with 'useful_word' and syn/ant for each 'useful_word'
#For each term we have a list of strings for its 'Antonyms' and 'Synonyms'
for i in range(len(test_array_df)):
    test_array_df.iloc[i,0] = useful_words[i]
    test_array_df['Synonyms'].loc[i] = raw_syn_list[i]
    test_array_df['Antonyms'].loc[i] = raw_ant_list[i]
test_array_df


In [ ]:
#ON HOLD FOR NOW
#For WordNet
useful_words = ['straight']
for word in useful_words: #pull words from a premade list to query WordNet
    driver = webdriver.Chrome()
    driver.get('http://wordnetweb.princeton.edu/perl/webwn?s=' + word + '&sub=Search+WordNet&o2=&o0=1&o8=1&o1=1&o7=&o5=&o9=&o6=&o3=&o4=&h=000000000000000000000')
    
    html_doc = driver.page_source # stores the source HTML code in the driver's page_source attribute
    soup = bs(html_doc, 'html.parser')
    syn = soup.find('a', {'href':"webwn?o2=&amp;o0=1&amp;o8=1&amp;o1=1&amp;o7=&amp;o5=&amp;o9=&amp;o6=&amp;o3=&amp;o4=&amp;s=" + word + "&amp;i=1&amp;h=00#c"})
    
    driver.quit()
    
print (syn)
    